In [ ]:
%%capture
!apt-get install tesseract-ocr
!pip install pytesseract Pillow discord

#@markdown <br><center><h3><font color=white><b>🛠️ INSTALL DEPENDENCIES 🛠️</b></h3>

In [ ]:
#@markdown <br><center><h3><font color=white><b>🔄 RUN 🤖 BOT 🔄</b></h3>
BOT_TOKEN = "" #@param {type:"string"}
CHANNEL_ID = "" #@param {type:"string"}
CHANNEL_ID = int(CHANNEL_ID)

#@markdown <br><center><a href="https://discordgsm.com/guide/how-to-get-a-discord-bot-token" target="_blank">GET DISCORD BOT TOKEN</a>
#@markdown <br><center><a href="https://support.discord.com/hc/en-us/articles/206346498-Where-can-I-find-my-User-Server-Message-ID" target="_blank">GET CHANNEl ID</a>

import json
import discord
from io import BytesIO
import pytesseract
from PIL import Image
import nest_asyncio
import sys
import os

# Function to apply nest_asyncio patch if necessary
def apply_nest_asyncio_patch():
    try:
        nest_asyncio.apply()
    except Exception as e:
        print(f"Error applying nest_asyncio patch: {e}")
        sys.exit(1)

def extract_text_from_image(image_file):
    try:
        img = Image.open(image_file)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"Error occurred: {e}"

def main():
    apply_nest_asyncio_patch()

    intents = discord.Intents.default()
    intents.messages = True
    intents.message_content = True

    client = discord.Client(intents=intents)

    @client.event
    async def on_ready():
        print(f'Logged in as {client.user}!')
        await client.change_presence(activity=discord.Game(name="Made by JuniorAlive"))

    @client.event
    async def on_message(message):
        if message.channel.id != CHANNEL_ID:
            return

        if message.attachments:
            for attachment in message.attachments:
                response = await attachment.read()
                if any(attachment.filename.endswith(ext) for ext in ['.png', '.jpg', '.jpeg', '.gif', '.bmp']):
                    image_to_text = extract_text_from_image(BytesIO(response))
                    extracted_text = image_to_text if image_to_text else "Failed to extract text"
                    if len(extracted_text) > 1900:
                        with open("output.txt", "w", encoding="utf-8") as text_file:
                            text_file.write(extracted_text)
                        with open("output.txt", "rb") as text_file:
                            await message.reply(file=discord.File(text_file, "output.txt"))
                    else:
                        await message.reply("```" + extracted_text.strip()[:1990] + "```")
                else:
                    await message.reply("Unsupported file type.")

    client.run(BOT_TOKEN)

if __name__ == "__main__":
    main()
